<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
# <font color='blue'>Data Science Academy</font>
# <font color='blue'>Projeto de Deploy com SageMaker, Lambda e API Gateway</font>
Acompanhe as instruções no capítulo do curso.

In [1]:
# Imports
import os
import boto3
import pandas as pd
import sagemaker
from sagemaker import get_execution_role
from sagemaker.inputs import TrainingInput
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
## Configurações Iniciais no SageMaker

In [2]:
# Define a região
dsa_region = boto3.Session().region_name
print(dsa_region)

us-east-2


In [3]:
# Define a role
dsa_role = get_execution_role()
print(dsa_role)

arn:aws:iam::890582101704:role/service-role/AmazonSageMaker-ExecutionRole-20240523T163775


In [4]:
# Cria sessão SageMaker
dsa_session = sagemaker.Session()
print(dsa_session)

## Carregando os Dados

In [5]:
# Cria o cliente s3
s3 = boto3.client("s3")

In [6]:
# Download do arquivo do bucket para o ambiente local do SageMaker
s3.download_file(f"dsa-projeto-890582101704", "dataset.csv", "dados_originais.csv")

In [7]:
# Carrega o arquivo como dataframe do pandas
df_dsa = pd.read_csv("./dados_originais.csv")

## Exploração Inicial dos Dados

In [8]:
# Visualiza as primeiras linhas
df_dsa.head()

,id,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007
1,1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007
2,2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007
3,3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN,2007
4,4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007


In [9]:
# Shape
df_dsa.shape

(344, 9)

In [10]:
# Total de valores ausentes
df_dsa.isna().sum()

id                    0
species               0
island                0
bill_length_mm        2
bill_depth_mm         2
flipper_length_mm     2
body_mass_g           2
sex                  11
year                  0
dtype: int64

In [11]:
# Total de duplicatas (complete cases)
df_dsa.duplicated().sum()

0

In [12]:
# Info
df_dsa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 344 non-null    int64  
 1   species            344 non-null    object 
 2   island             344 non-null    object 
 3   bill_length_mm     342 non-null    float64
 4   bill_depth_mm      342 non-null    float64
 5   flipper_length_mm  342 non-null    float64
 6   body_mass_g        342 non-null    float64
 7   sex                333 non-null    object 
 8   year               344 non-null    int64  
dtypes: float64(4), int64(2), object(3)
memory usage: 24.3+ KB


In [13]:
# Total de valores únicos
df_dsa.nunique()

id                   344
species                3
island                 3
bill_length_mm       164
bill_depth_mm         80
flipper_length_mm     55
body_mass_g           94
sex                    2
year                   3
dtype: int64

In [14]:
# Resumo estatístico
df_dsa.describe()

,id,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,year
count,344.000000,342.000000,342.000000,342.000000,342.000000,344.000000
mean,171.500000,43.921930,17.151170,200.915205,4201.754386,2008.031977
std,99.448479,5.459584,1.974793,14.061714,801.954536,0.816464
min,0.000000,32.100000,13.100000,172.000000,2700.000000,2007.000000
25%,85.750000,39.225000,15.600000,190.000000,3550.000000,2007.000000
50%,171.500000,44.450000,17.300000,197.000000,4050.000000,2008.000000
75%,257.250000,48.500000,18.700000,213.000000,4750.000000,2009.000000
max,343.000000,59.600000,21.500000,231.000000,6300.000000,2009.000000


## Limpeza, Codificação e Preparação dos Dados

In [15]:
# Colunas
df_dsa.columns

Index(['id', 'species', 'island', 'bill_length_mm', 'bill_depth_mm',
       'flipper_length_mm', 'body_mass_g', 'sex', 'year'],
      dtype='object')

In [16]:
print("Categorias na Variável 'species': ", df_dsa['species'].unique())

Categorias na Variável 'species':  ['Adelie' 'Gentoo' 'Chinstrap']


In [17]:
print("Categorias na Variável 'island': ", df_dsa['island'].unique())

Categorias na Variável 'island':  ['Torgersen' 'Biscoe' 'Dream']


In [18]:
# Vamos fazer o drop de variáveis que não serão relevantes para o projeto
dados_limpos = df_dsa.drop(columns = ['id', 'island', 'sex', 'year'])

In [19]:
# Info
dados_limpos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    object 
 1   bill_length_mm     342 non-null    float64
 2   bill_depth_mm      342 non-null    float64
 3   flipper_length_mm  342 non-null    float64
 4   body_mass_g        342 non-null    float64
dtypes: float64(4), object(1)
memory usage: 13.6+ KB


In [20]:
# Vamos fazer o drop de valores ausentes
dados_limpos = dados_limpos.dropna(subset = ['bill_length_mm'])

In [21]:
dados_limpos.isna().sum()

species              0
bill_length_mm       0
bill_depth_mm        0
flipper_length_mm    0
body_mass_g          0
dtype: int64

In [22]:
dados_limpos.head()

,species,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
0,Adelie,39.1,18.7,181.0,3750.0
1,Adelie,39.5,17.4,186.0,3800.0
2,Adelie,40.3,18.0,195.0,3250.0
4,Adelie,36.7,19.3,193.0,3450.0
5,Adelie,39.3,20.6,190.0,3650.0


In [23]:
# Label encoding na variável alvo
dados_limpos['species'] = dados_limpos['species'].replace('Adelie', 0)
dados_limpos['species'] = dados_limpos['species'].replace('Gentoo', 1)
dados_limpos['species'] = dados_limpos['species'].replace('Chinstrap', 2)

In [24]:
# Shuffle (embaralhamento dos dados)
dados_limpos = dados_limpos.sample(frac = 1).reset_index(drop = True)

In [25]:
dados_limpos.sample(10)

,species,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
219,0,37.5,18.9,179.0,2975.0
274,0,40.6,18.8,193.0,3800.0
266,0,36.5,16.6,181.0,2850.0
312,1,45.1,14.5,215.0,5000.0
124,1,49.6,15.0,216.0,4750.0
228,2,45.7,17.0,195.0,3650.0
275,2,49.7,18.6,195.0,3600.0
168,0,39.6,18.8,190.0,4600.0
114,2,49.3,19.9,203.0,4050.0
307,1,50.1,15.0,225.0,5000.0


## Divisão dos Dados em Treino e Teste e Armazenamento no S3

In [26]:
# Garante que somente as colunas do shape do dataframe serão usadas 
dados_limpos.columns = range(dados_limpos.shape[1])

In [27]:
dados_limpos.sample(5)

,0,1,2,3,4
169,0,41.8,19.4,198.0,4450.0
295,0,38.6,17.0,188.0,2900.0
256,0,36.2,17.2,187.0,3150.0
185,1,47.8,15.0,215.0,5650.0
326,2,50.9,17.9,196.0,3675.0


In [28]:
# Divisão em treino e validação com proporção 80/20
dsa_dados_treino, dsa_dados_valid = train_test_split(dados_limpos, test_size = 0.2, random_state = 42)

In [29]:
dsa_dados_treino.shape

(273, 5)

In [30]:
dsa_dados_valid.shape

(69, 5)

In [31]:
# Nome do bucket S3
bucket_name = 'dsa-projeto-890582101704'

In [32]:
# Salvando e enviando o arquivo de treino
dsa_dados_treino.to_csv('dados_treino.csv', header = False, index = False)
key = 'dsa/treino/dados_treino'
url = 's3://{}/{}'.format(bucket_name, key)
boto3.Session().resource('s3').Bucket(bucket_name).Object(key).upload_file('dados_treino.csv')

In [33]:
# Salvando e enviando o arquivo de validação
dsa_dados_valid.to_csv('dados_valid.csv', header = False, index = False)
key = 'dsa/val/dados_valid'
url = 's3://{}/{}'.format(bucket_name, key)
boto3.Session().resource('s3').Bucket(bucket_name).Object(key).upload_file('dados_valid.csv')

## Construção do Modelo

In [34]:
# Nome do bucket S3
bucket_name = 'dsa-projeto-890582101704'

In [35]:
# Nome para salvar o modelo
key = 'dsa_model/xgbmodel'

In [36]:
# Local no S3 para salvar o modelo
s3_output_location = f's3://{bucket_name}/{key}'

In [37]:
# Define a URI da imagem do modelo XGBoost no SageMaker
image_uri = sagemaker.image_uris.retrieve("xgboost", dsa_region, version = "1.5-1")

In [38]:
# Cria o estimador XGBoost
dsa_xgbmodel = sagemaker.estimator.Estimator(image_uri = image_uri,
                                             role = dsa_role,
                                             instance_count = 1,
                                             instance_type = 'ml.m4.xlarge',
                                             volume_size = 5,
                                             output_path = s3_output_location,
                                             sagemaker_session = dsa_session)

In [39]:
# Configura os hiperparâmetros
dsa_xgbmodel.set_hyperparameters(max_depth = 5,
                                 gamma = 4,
                                 min_child_weight = 6,
                                 objective = 'multi:softmax',
                                 num_class = 3,
                                 num_round = 8)

Descrição dos hiperparâmetros configurados no modelo dsa_xgbmodel para o algoritmo XGBoost:

**max_depth (5)**: Define a profundidade máxima de cada árvore de decisão no modelo. Controla a complexidade do modelo, evitando que ele se torne excessivamente complexo e, portanto, sujeito a overfitting. Um valor de 5 permite que cada árvore tenha até 5 níveis de profundidade.

**gamma (4)**: Representa o valor mínimo de redução na função de perda para que uma nova divisão em uma árvore seja feita. Um valor maior torna o modelo mais conservador, pois exige maior melhora na função objetivo para criar novas divisões.

**min_child_weight (6)**: Especifica o peso mínimo de somatório de instâncias em um nó filho para que ele seja subdividido. Este parâmetro ajuda a controlar o underfitting. Um valor maior exige que cada nó tenha mais instâncias, resultando em árvores mais simples.

**objective ('multi:softmax')**: Define a função objetivo para o modelo. Aqui, foi configurado como multi:softmax, que é usado para problemas de classificação multiclasse. Ele gera como saída a classe prevista diretamente, em vez de probabilidades.

**num_class (3)**: Indica o número de classes distintas no problema de classificação. Nesse caso, o modelo está configurado para lidar com 3 classes diferentes.
<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
**num_round (8)**: Refere-se ao número de iterações ou rodadas de boosting. Cada rodada adiciona uma nova árvore ao ensemble para melhorar as previsões do modelo. Com 8 rodadas, o modelo será atualizado iterativamente até esse limite.

## Treinamento do Modelo

In [40]:
# Define o caminho para os dados
dados_treino_dsa = 's3://{}/{}'.format(bucket_name, 'dsa/treino/dados_treino')
dados_valid_dsa = 's3://{}/{}'.format(bucket_name, 'dsa/val/dados_val')

In [41]:
dados_treino_dsa

's3://dsa-projeto-890582101704/dsa/treino/dados_treino'

In [42]:
dados_valid_dsa

's3://dsa-projeto-890582101704/dsa/val/dados_val'

In [43]:
# Cria os inputs
input_treino = TrainingInput(s3_data = dados_treino_dsa, content_type = 'text/csv')
input_val = TrainingInput(s3_data = dados_valid_dsa, content_type = 'text/csv')

In [44]:
# Define os canais para os dados
canais_dados = {'train': input_treino, 'validation': input_val}

In [45]:
%%time

dsa_xgbmodel.fit(inputs = canais_dados)

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2024-12-27-19-27-52-837


2024-12-27 19:27:54 Starting - Starting the training job...
2024-12-27 19:28:09 Starting - Preparing the instances for training...
2024-12-27 19:28:55 Downloading - Downloading the training image......
2024-12-27 19:29:51 Training - Training image download completed. Training in progress.../miniconda3/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
[2024-12-27 19:30:06.850 ip-10-0-247-188.us-east-2.compute.internal:6 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2024-12-27 19:30:06.872 ip-10-0-247-188.us-east-2.compute.internal:6 INFO profiler_config_parser.py:111] User has disabled profiler.
[2024-12-27:19:30:07:INFO] Imported framework sagemaker_xgboost_container.training
[2024-12-27:19:30:07:INFO] Failed to parse hyperparameter objective value multi:softmax to Json.
Returning th

## Deploy do Modelo com SageMaker

In [46]:
%%time

xgb_predictor = dsa_xgbmodel.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2024-12-27-19-36-29-232
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2024-12-27-19-36-29-232
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2024-12-27-19-36-29-232


------!CPU times: user 64.8 ms, sys: 3.14 ms, total: 68 ms
Wall time: 3min 32s


Ao executar a linha acima, o modelo treinado (dsa_xgbmodel) é implantado em um endpoint gerenciado pelo Amazon SageMaker para inferência em tempo real. Aqui está o que acontece:

**Criação do Endpoint**: O SageMaker cria um endpoint RESTful onde o modelo ficará disponível para receber solicitações de previsão.

**Configuração da Infraestrutura**: É provisionada uma instância do tipo ml.m4.xlarge para hospedar o modelo. Essa instância é dimensionada com base nos requisitos computacionais do modelo.

**Inicialização do Modelo**: O modelo treinado é carregado na instância provisionada, junto com o ambiente necessário para sua execução, como o container do XGBoost.

**Atribuição ao Objeto xgb_predictor**: O objeto xgb_predictor é criado e serve como uma interface para interagir com o endpoint. Ele permite enviar dados para predição e receber as respostas diretamente no ambiente de desenvolvimento.

Este processo disponibiliza o modelo em um ambiente escalável e gerenciado, pronto para realizar inferências em tempo real a partir de requisições enviadas ao endpoint.

Agora vamos criar a API.

# Fim